Web Scraping Request

In [20]:
!pip install httpx parsel loguru

In [21]:
!pip install asyncio

In [52]:
import requests
from bs4 import BeautifulSoup
 
 
url = 'https://www.zillow.com/orange-county-ny/luxury-homes/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22west%22%3A-75.95072266015626%2C%22east%22%3A-72.75919433984376%2C%22south%22%3A40.5702843184022%2C%22north%22%3A42.19609605347439%7D%2C%22usersSearchTerm%22%3A%22Orange%20County%2C%20NY%22%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A1290%2C%22regionType%22%3A4%7D%5D%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22priced%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A9%7D'
reqs = requests.get(url)
soup = BeautifulSoup(reqs.text, 'html.parser')
 
urls = []
for link in soup.find_all("script"):
    print(link.get("href"))

None


In [54]:
import asyncio
from typing import List
import httpx
import json
from parsel import Selector
import nest_asyncio
nest_asyncio.apply()

async def fetch(url: str, client: httpx.AsyncClient):
    """Fetches a single URL asynchronously."""
    response = await client.get(url)
    response.raise_for_status()  # Raises an exception if the response is not successful
    return response

async def parse_response(response: httpx.Response):
    """Parses property data from the response."""
    selector = Selector(response.text)
    data = selector.css("script#__NEXT_DATA__::text").get()
    if data:
        data = json.loads(data)
        property_data = json.loads(data["props"]["pageProps"]["componentProps"]["gdpClientCache"])
        property_data = property_data[list(property_data)[0]]['property']
    else:
        data = selector.css("script#hdpApolloPreloadedData::text").get()
        data = json.loads(json.loads(data)["apiCache"])
        property_data = next(
            v["property"] for k, v in data.items() if "ForSale" in k
        )
    return property_data

async def scrape_properties(urls: List[str]):
    """Scrapes property pages for property data."""
    async with httpx.AsyncClient(http2=True, headers={
        "accept-language": "en-US,en;q=0.9",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
        "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        "accept-encoding": "gzip, deflate, br",
    }) as client:
        tasks = [fetch(url, client) for url in urls]
        responses = await asyncio.gather(*tasks)
        results = [await parse_response(response) for response in responses]
    return results

async def main():
    urls = ["https://www.zillow.com/homedetails/Maples-Rd-Middletown-NY-10940/163621205_zpid/"]
    data = await scrape_properties(urls)
    print(json.dumps(data, indent=2))

if __name__ == "__main__":
    asyncio.run(main())


[
  {
    "listingDataSource": "Phoenix",
    "zpid": 163621205,
    "city": "Middletown",
    "state": "NY",
    "homeStatus": "FOR_SALE",
    "address": {
      "streetAddress": " Maples Road",
      "city": "Middletown",
      "state": "NY",
      "zipcode": "10940",
      "neighborhood": null,
      "community": null,
      "subdivision": null
    },
    "isListingClaimedByCurrentSignedInUser": false,
    "isCurrentSignedInAgentResponsible": false,
    "bedrooms": null,
    "bathrooms": null,
    "price": 7595000,
    "yearBuilt": null,
    "streetAddress": " Maples Road",
    "zipcode": "10940",
    "isCurrentSignedInUserVerifiedOwner": false,
    "propertyUpdatePageLink": null,
    "moveHomeMapLocationLink": null,
    "propertyEventLogLink": null,
    "editPropertyHistorylink": null,
    "collections": {
      "modules": [
        {
          "name": "Similar homes",
          "placement": "NEIGHBORHOOD",
          "propertyDetails": [
            {
              "miniCardPhotos"